# Project 1
## Harris Dupre
## Summer 2020, DATA 643


#### This system recommends pizzerias to hungry people.


In [384]:
import pandas as pd;
import numpy as np;
import math
from itertools import combinations_with_replacement

# create data and load into dataframe
d = {'Rockys':[1,np.nan,2,5,3,2,3,2,4,3], 'Librettos':[2,4,3,np.nan,4,3,np.nan,2,np.nan,1], 'Little Italy':[3,2,2,4,np.nan,3,2,4,1,5], 'Joes':[4,np.nan,5,3,2,np.nan,2,5,4,2], 'Cascarinos':[5,5,4,np.nan,4,3,4,2,5,np.nan]}
df = pd.DataFrame(data=d, index = ['Harris', 'Jess', 'Madelyn','Paul','Rajiv','Cathy','Jamie','Clara','Ben','Ming'])

df

,Rockys,Librettos,Little Italy,Joes,Cascarinos
Harris,1.0,2.0,3.0,4.0,5.0
Jess,NaN,4.0,2.0,NaN,5.0
Madelyn,2.0,3.0,2.0,5.0,4.0
Paul,5.0,NaN,4.0,3.0,NaN
Rajiv,3.0,4.0,NaN,2.0,4.0
Cathy,2.0,3.0,3.0,NaN,3.0
Jamie,3.0,NaN,2.0,2.0,4.0
Clara,2.0,2.0,4.0,5.0,2.0
Ben,4.0,NaN,1.0,4.0,5.0
Ming,3.0,1.0,5.0,2.0,NaN


### Breaking data into separate trianing and test datasets

In [385]:
# create copies of df to work with as training and test sets
train_set_df = df.copy()
df_copy = df.copy()

# create a dataframe of randomly selected test values from each item
test_values = [df_copy.at["Madelyn", "Rockys"],df_copy.at["Clara", "Librettos"],df_copy.at["Cathy", "Little Italy"],df_copy.at["Harris", "Joes"],df_copy.at["Rajiv", "Cascarinos"]]
test_index = ['MadelynRockys', 'ClaraLibrettos', 'CathyLittleItaly','HarrisJoes','RajivCascarinos']
test_set_df = pd.DataFrame(data=test_values,index=test_index,columns=["Rating"])

# remove test set values from the training set df by assigning to NaN
train_set_df.at["Madelyn", "Rockys"] = np.NaN
train_set_df.at["Clara", "Librettos"] = np.NaN
train_set_df.at["Cathy", "Little Italy"] = np.NaN
train_set_df.at["Harris", "Joes"] = np.NaN
train_set_df.at["Rajiv", "Cascarinos"] = np.NaN

train_set_df

,Rockys,Librettos,Little Italy,Joes,Cascarinos
Harris,1.0,2.0,3.0,NaN,5.0
Jess,NaN,4.0,2.0,NaN,5.0
Madelyn,NaN,3.0,2.0,5.0,4.0
Paul,5.0,NaN,4.0,3.0,NaN
Rajiv,3.0,4.0,NaN,2.0,NaN
Cathy,2.0,3.0,NaN,NaN,3.0
Jamie,3.0,NaN,2.0,2.0,4.0
Clara,2.0,NaN,4.0,5.0,2.0
Ben,4.0,NaN,1.0,4.0,5.0
Ming,3.0,1.0,5.0,2.0,NaN


In [386]:
test_set_df

,Rating
MadelynRockys,2.0
ClaraLibrettos,2.0
CathyLittleItaly,3.0
HarrisJoes,4.0
RajivCascarinos,4.0


### Calculating the raw average

In [387]:
# calculate the raw average for the training dataset
mean_train = train_set_df.stack().mean()

round(mean_train,3) 

3.167

### Finding the RSME for the training and test sets

In [388]:
def subtract_mean_and_square(x):
    # Function will subtract the mean of the training set mean_train and square the result for each cell
    return (x-mean_train)**2

rsme_train = math.sqrt(train_set_df.apply(subtract_mean_and_square).stack().mean())
rsme_test = math.sqrt(test_set_df.apply(subtract_mean_and_square).mean())

round(rsme_train,3)

1.258

In [389]:
round(rsme_test,3)

0.91

### Calculating the user bias and item bias values

In [390]:
user_bias_df = train_set_df.mean(axis=1) - mean_train

user_bias_df.round(3)

Harris    -0.417
Jess       0.500
Madelyn    0.333
Paul       0.833
Rajiv     -0.167
Cathy     -0.500
Jamie     -0.417
Clara      0.083
Ben        0.333
Ming      -0.417
dtype: float64

In [391]:
item_bias_df = train_set_df.mean() - mean_train

item_bias_df.round(3)

Rockys         -0.292
Librettos      -0.333
Little Italy   -0.292
Joes            0.119
Cascarinos      0.833
dtype: float64

### Calculating the baseline predictors

In [392]:
train_set_blp = train_set_df.copy()

# save the column and index names of train_set_blp
columns_blp = train_set_blp.columns
index_blp = train_set_blp.index

# set columns and indexes to integers for easy loop iteration
train_set_blp.columns = [0,1,2,3,4]
train_set_blp.index = [0,1,2,3,4,5,6,7,8,9]
item_bias_df.index = [0,1,2,3,4]
user_bias_df.index = [0,1,2,3,4,5,6,7,8,9]

# loop through and sum each combination of user bias and item bias and the raw average
for i in range(10):
    for j in range(5):
        train_set_blp.at[i,j] = mean_train + user_bias_df.at[i] + item_bias_df.at[j] 

# reset train_set_blp column and row names
train_set_blp.columns = columns_blp
train_set_blp.index = index_blp

# display baseline predictors for every user-item combination
train_set_blp 

,Rockys,Librettos,Little Italy,Joes,Cascarinos
Harris,2.458333,2.416667,2.458333,2.869048,3.583333
Jess,3.375000,3.333333,3.375000,3.785714,4.500000
Madelyn,3.208333,3.166667,3.208333,3.619048,4.333333
Paul,3.708333,3.666667,3.708333,4.119048,4.833333
Rajiv,2.708333,2.666667,2.708333,3.119048,3.833333
Cathy,2.375000,2.333333,2.375000,2.785714,3.500000
Jamie,2.458333,2.416667,2.458333,2.869048,3.583333
Clara,2.958333,2.916667,2.958333,3.369048,4.083333
Ben,3.208333,3.166667,3.208333,3.619048,4.333333
Ming,2.458333,2.416667,2.458333,2.869048,3.583333


### Calculate the RSME for the baseline predictors for training and test data

In [393]:
train_set_blp_rsme = pd.DataFrame()

# set columns and indexes to integers for easy loop iteration
train_set_blp.columns = [0,1,2,3,4]
train_set_blp.index = [0,1,2,3,4,5,6,7,8,9]
train_set_df.columns = [0,1,2,3,4]
train_set_df.index = [0,1,2,3,4,5,6,7,8,9]

for i in range(10):
    for j in range(5):
        train_set_blp_rsme.at[i,j] = (train_set_df.at[i,j] - train_set_blp.at[i,j])**2

train_set_blp_rsme

,0,1,2,3,4
0,2.126736,0.173611,0.293403,NaN,2.006944
1,NaN,0.444444,1.890625,NaN,0.250000
2,NaN,0.027778,1.460069,1.907029,0.111111
3,1.668403,NaN,0.085069,1.252268,NaN
4,0.085069,1.777778,NaN,1.252268,NaN
5,0.140625,0.444444,NaN,NaN,0.250000
6,0.293403,NaN,0.210069,0.755244,0.173611
7,0.918403,NaN,1.085069,2.660006,4.340278
8,0.626736,NaN,4.876736,0.145125,0.444444
9,0.293403,2.006944,6.460069,0.755244,NaN


In [394]:
# finishe RSME calculation using baseline predictor values
train_blp_rsme = math.sqrt(train_set_blp_rsme.stack().mean())

# display training set baseline predictor RSME
round(train_blp_rsme,3)

1.102

In [395]:
# extract baseline predictor values for test from train_set_blp
df2_copy = train_set_blp.copy()
df2_copy.columns = columns_blp
df2_copy.index = index_blp

# add a column of baseline predictor values to test dataframe
test_blp_values = [df2_copy.at["Madelyn", "Rockys"],df2_copy.at["Clara", "Librettos"],df2_copy.at["Cathy", "Little Italy"],df2_copy.at["Harris", "Joes"],df2_copy.at["Rajiv", "Cascarinos"]]
test_set_df['Baseline Predictor'] = test_blp_values

# add an RMSE calculation column by squaring the result of subtracting the rating from the baseline predictor value of the test combinations
test_set_df['RMSE Calc'] = (test_set_df['Rating'] - test_set_df['Baseline Predictor'])**2

In [396]:
test_set_df

,Rating,Baseline Predictor,RMSE Calc
MadelynRockys,2.0,3.208333,1.460069
ClaraLibrettos,2.0,2.916667,0.840278
CathyLittleItaly,3.0,2.375000,0.390625
HarrisJoes,4.0,2.869048,1.279053
RajivCascarinos,4.0,3.833333,0.027778


In [397]:
# complete the RSME calculation on the RSME Calc column
test_blp_rsme = math.sqrt(test_set_df['RMSE Calc'].mean())
# display test set baseline predictor RSME
round(test_blp_rsme,3)

0.894

## Summary

Our initial training RSME was 1.258 while the initial test RSME was 0.91.

Baseline predictor RSME was 1.102 for the training set and 0.894 for the test set.

In [398]:
percent_improve_training = (1 - (1.102/1.258))*100
percent_improve_test = (1 - (0.894/0.91))*100

print(round(percent_improve_training,3),"%")
print(round(percent_improve_test,3),"%")

12.401 %
1.758 %


Our predictions were 12% better by using baseline predictor RSME for the training set while only 1.76% better by using the baseline predictor RSME for the test set.